In [1]:
try:
    from isaacgym import gymtorch
    from isaacgym import gymapi
except ImportError:
    pass
from bc_algos.dataset.robomimic import RobomimicDataset
from bc_algos.envs.robosuite import RobosuiteEnv
from bc_algos.dataset.isaac_gym import IsaacGymDataset
from bc_algos.envs.isaac_gym import IsaacGymEnv
from bc_algos.utils.constants import Modality
import bc_algos.utils.constants as Constants
import omegaconf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import cv2

# change this to test different simulator environments
type = Constants.EnvType.ISAAC_GYM

Importing module 'gym_38' (/home/niksrid/IsaacGym/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_38.so)
Setting GYM_USD_PLUG_INFO_PATH to /home/niksrid/IsaacGym/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 2.3.0+cu118
Device count 4
/home/niksrid/IsaacGym/isaacgym/python/isaacgym/_bindings/src/gymtorch


Using /home/niksrid/.cache/torch_extensions/py38_cu118 as PyTorch extensions root...
Emitting ninja build file /home/niksrid/.cache/torch_extensions/py38_cu118/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module gymtorch...
/home/niksrid/miniconda3/envs/bc_algos/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Dataset Init

In [2]:
obs_key_to_modality = {
    "robot0_eef_pos": Modality.LOW_DIM,
    "robot0_eef_quat": Modality.LOW_DIM,
    "agentview_image": Modality.RGB
}
obs_group_to_key = {
    "obs": ["robot0_eef_pos", "robot0_eef_quat", "agentview_image"],
    "goal": ["agentview_image"]
}
action_key = "actions"

if type == Constants.EnvType.ROBOSUITE:
    path = "../datasets/test/square_ph.hdf5"
    demo_id = "demo_0"

elif type == Constants.DatasetType.ISAAC_GYM:
    path = "../datasets/dataset_v5"
    demo_id = 95

In [3]:
if type == Constants.EnvType.ROBOSUITE:
    dataset = RobomimicDataset(
        path=path,
        obs_key_to_modality=obs_key_to_modality,
        obs_group_to_key=obs_group_to_key,
        action_key=action_key,
        history=0,
        action_chunk=1,
        pad_history=False,
        pad_action_chunk=False,
        get_pad_mask=False,
        demo_ids=[demo_id],
    )
if type == Constants.EnvType.ISAAC_GYM:
    dataset = IsaacGymDataset(
        path=path,
        obs_key_to_modality=obs_key_to_modality,
        obs_group_to_key=obs_group_to_key,
        action_key=action_key,
        history=0,
        action_chunk=1,
        pad_history=False,
        pad_action_chunk=False,
        get_pad_mask=False,
        demo_ids=[demo_id],
    )

caching sequences: 100%|██████████| 1/1 [00:00<00:00, 25.65demo/s]


### Environment Init

In [4]:
if type == Constants.EnvType.ROBOSUITE:
    env_meta = json.loads(dataset.hdf5_file["data"].attrs["env_args"])
    env = RobosuiteEnv(
        env_name=env_meta["env_name"],
        obs_key_to_modality=obs_key_to_modality,
        render=False,
        use_image_obs=True,
        use_depth_obs=False,
        **env_meta["env_kwargs"],
    )
elif type == Constants.EnvType.ISAAC_GYM:
    env_cfg_path = "../config/isaac_gym_env.json"
    config = omegaconf.OmegaConf.create(json.load(open(env_cfg_path, "r")))
    env = IsaacGymEnv(
        env_name=config.task.name,
        obs_key_to_modality=obs_key_to_modality,
        render=False,
        use_image_obs=True,
        use_depth_obs=False,
        config=config,
    )

2024-06-13 04:46:52,947 - INFO - logger - logger initialized
<unknown>:3: DeprecationWarning: invalid escape sequence \*
/home/niksrid/IsaacGym/isaacgym/python/isaacgym/torch_utils.py:135: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def get_axis_params(value, axis_idx, x_value=0., dtype=np.float, n_dims=3):


Error: FBX library failed to load - importing FBX data will not succeed. Message: No module named 'fbx'
FBX tools must be installed from https://help.autodesk.com/view/FBX/2020/ENU/?guid=FBX_Developer_Help_scripting_with_python_fbx_installing_python_fbx_html


/home/niksrid/miniconda3/envs/bc_algos/lib/python3.8/site-packages/networkx/classes/graph.py:23: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping
/home/niksrid/miniconda3/envs/bc_algos/lib/python3.8/site-packages/networkx/classes/reportviews.py:95: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Mapping, Set, Iterable
/home/niksrid/miniconda3/envs/bc_algos/lib/python3.8/site-packages/networkx/readwrite/graphml.py:346: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wi

Not connected to PVD
+++ Using GPU PhysX
Physics Engine: PhysX
Physics Device: cuda:0
GPU Pipeline: enabled


/home/niksrid/IsaacGym/IsaacGymEnvs/isaacgymenvs/tasks/mental_models_task.py:72: DeprecationWarning: an integer is required (got type isaacgym._bindings.linux-x86_64.gym_38.DofDriveMode).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  asset_options.default_dof_drive_mode = gymapi.DOF_MODE_EFFORT


num franka bodies:  16
num franka dofs:  9
[Warning] [carb.gym.plugin] Acquiring DOF force tensor, but no actors have DOF force sensors enabled.
[Warning] [carb.gym.plugin] -> Enabled DOF force sensors for all eligible actors.


### Test Environment

In [5]:
# initialize environment to same simulation state at the beginning of the demo
if type == Constants.EnvType.ROBOSUITE:
    xml = dataset.hdf5_file[f"data/{demo_id}"].attrs["model_file"]
    env.load_env(xml=xml)
    init_state = dataset.hdf5_file[f"data/{demo_id}/states"][0]
    env.reset_to(state=init_state)
elif type == Constants.EnvType.ISAAC_GYM:
    run = pd.read_pickle(dataset.demo_id_to_run_path(demo_id=demo_id))
    metadata = run["metadata"]
    cubes_pos = run["obs"]["cubes_pos"][0]
    cubes_quat = run["obs"]["cubes_quat"][0]
    cubes_pose = np.concatenate([cubes_pos, cubes_quat], axis=-1)
    metadata.update({
        "block_init_pose": cubes_pose,
        "start_q": run["obs"]["q"][0],
    })
    env.reset_to(state=metadata)

In [ ]:
# compare reference and executed trajectories visually
demo_length = dataset.demo_len(demo_id=demo_id)
demo = dataset.load_demo(demo_id=demo_id)
for i in range(demo_length):
    frame = dataset.seq_from_timstep(demo_id=demo_id, demo=demo, t=i)
    action = frame[action_key][0]
    ref_image = np.transpose(frame["obs"]["agentview_image"][0], (1, 2, 0))
    img = env.render()
    # fig, axs = plt.subplots(1, 2)
    # axs[0].imshow(ref_image)
    # axs[0].set_title("Reference")
    # axs[1].imshow(img)
    # axs[1].set_title("Execution")
    # plt.savefig(f"../outputs/test_env/{i}.jpg")
    # plt.draw()
    # plt.close()
    cv2.imwrite(f"../outputs/test_env/{i}.jpg", cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    env.step(action=action)